# Example: convert DHARMA LES output to DEPHY format

Code to read DHARMA LES output files and write to DEPHY format (NetCDF)

Contributed by Ann Fridlind from NASA/GISS

## Import libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import netCDF4
import datetime as dt
from netCDF4 import Dataset

## Specify directory locations

If on the ARM JupyterHub, it is recommended to create and specify a local directory that is outside of the COMBLE-MIP repository to upload raw model output files in your model's format. 

Processed domain-mean outputs are invited for commit to the GitHub repository on a user-specified branch under /comble-mip/output_les/YOUR_MODEL_NAME/sandbox/YOUR_RUN_NAME. These can be committed and removed at any time.

It is requested to name your baseline small-domain run directory as 'Lx25km_dx100m' (in place of YOUR_RUN_NAME), so that it can readily be automatically compared with other runs using the same test specification.

In [2]:
# specify start time of simulation and simulation name
start_dtime = '2020-03-12 22:00:00.0'

# specify input data directory name, traceable to source machine, and
# specify output file name (see file naming convention in TOC)
#my_readdir = 'case0313_diagn_ice25_miz_cfmip_dx100'
#my_outfile = 'DHARMA_Lx25km_dx100_FixN.nc'

my_readdir = 'case0313_progn_ice25_miz_cfmip_dx100'
my_outfile = 'DHARMA_Lx25km_dx100_ProgNa.nc'

#my_readdir = 'dx200_swoff_z0fix1'
#my_outfile = 'DHARMA_Lx25km_dx200_z0_4.nc'

# specify local source directories (additional subdirectories if restart was required)
my_rundir = '/data/home/fridlind/dharma/devel/' + my_readdir + '/'
## my_rundir = '/data/home/floriantornow/dharma/sandbox/' + my_readdir + '/'
my_outdirs = sorted([f for f in os.listdir(my_rundir) if not f.startswith('.')], key=str.lower)
print(my_outdirs)

# specify Github scratch directory where processed model output will be committed
my_gitdir = '../../output_les/dharma/devel/'

['0-2h', '02h-16h20m', '16h20m-20h']


## Read DHARMA data

### Read set-up parameters

Note: ERROR 1: PROJ... message can be ignored here.

In [3]:
# read in DHARMA parameter settings
input_filename = my_rundir + my_outdirs[0] + '/dharma.cdf'
dharma_params = xr.open_dataset(input_filename)
print(input_filename)

# check if the run contains ice variables
do_ice = bool(dharma_params['Cond'].do_ice)
print('do_ice = ',do_ice)

# check for prognostic aerosol
do_progNa = bool(dharma_params['Cond'].do_prog_na)
print('do_progNa = ',do_progNa)

# full parameter list
dharma_params

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/0-2h/dharma.cdf
do_ice =  True
do_progNa =  True


<xarray.Dataset>
Dimensions:      ()
Data variables: (12/26)
    geometry     int32 ...
    timing       int32 ...
    options      int32 ...
    zstretch     int32 ...
    assimilate   int32 ...
    restart      int32 ...
    ...           ...
    Radiation    int32 ...
    Surface      int32 ...
    Subsidence   int32 ...
    Turb         int32 ...
    InvAnalysis  int32 ...
    carma        int32 ...

### Read domain-mean profiles

In [4]:
# concatenate DHARMA domain-mean instantaneous profiles and take 10-min average:
# resample-average before concatenating and removing duplicates
for index, elem in enumerate(my_outdirs):
    input_filename = my_rundir + elem + '/dharma.soundings.cdf'
    print(input_filename)
    if index==0:
        dharma_snds = xr.open_dataset(input_filename)
        dharma_snds['time'] = pd.to_datetime(dharma_snds['time'].values, unit='s', origin=pd.Timestamp(start_dtime))
        dharma_snds = dharma_snds.resample(time="600S",closed="right",label="right").mean()
    else:
        dharma_snds2 = xr.open_dataset(input_filename)
        dharma_snds2['time'] = pd.to_datetime(dharma_snds2['time'].values, unit='s', origin=pd.Timestamp(start_dtime))
        dharma_snds2 = dharma_snds2.resample(time="600S",closed="right",label="right").mean()
        dharma_snds = xr.concat([dharma_snds,dharma_snds2],dim='time')
dharma_snds = dharma_snds.drop_duplicates('time',keep='first')
dharma_snds

/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/0-2h/dharma.soundings.cdf
/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/02h-16h20m/dharma.soundings.cdf
/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/16h20m-20h/dharma.soundings.cdf


<xarray.Dataset>
Dimensions:         (zt: 159, zw: 160, time: 121)
Coordinates:
  * zt              (zt) float32 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
  * zw              (zw) float32 0.0 20.0 45.0 75.0 ... 6.89e+03 6.95e+03 7e+03
  * time            (time) datetime64[ns] 2020-03-12T22:00:00 ... 2020-03-13T...
Data variables: (12/389)
    jact            (time, zt) float32 0.4543 0.5679 0.6814 ... 1.363 1.136
    jacw            (time, zw) float32 0.3975 0.5111 0.6246 ... 1.59 1.249 1.704
    rhobar          (time, zt) float32 1.399 1.396 1.391 ... 0.5989 0.594 0.5901
    u               (time, zt) float32 3.225 3.454 3.719 ... 4.824 4.85 4.877
    u2              (time, zt) float32 5.199 5.965 6.917 ... 11.64 11.77 11.9
    Su_rk           (time, zt) float32 0.0 0.0 0.0 ... 0.0003689 0.0003783
    ...              ...
    hw              (time, zw) float32 0.0 0.0 0.0 ... -6.972e-08 -1.138e-08 0.0
    qw              (time, zw) float32 0.0 0.0 0.0 ... 1.465e-10 2.31e-11 0.0
    txz_tot         (time, zw) float32 -0.09923 -0.07866 ... 4.921e-07 0.0
    tyz_tot         (time, zw) float32 0.4973 0.3938 0.1923 ... -2.75e-06 0.0
    qhz_tot         (time, zw) float32 -7.833e-05 -6.29e-05 ... -1.138e-08 0.0
    qqz_tot         (time, zw) float32 2.53e-06 1.956e-06 ... 2.31e-11 0.0
Attributes:
    nx:        256
    ny:        256
    nz:        159
    L_x:       25600.0
    L_y:       25600.0
    H:         7000.0
    theta_00:  247.95700073242188

### Profile unit conversions and sundry

In [5]:
# create a dummy sounding and initialize some variables needed
dummy_snd = dharma_snds['qc']*0.
nz = dharma_params['geometry'].nz
dz = dharma_snds['zw'].data[1:nz+1]-dharma_snds['zw'].data[0:nz]
cp = 1004.
Lhe = 2.50e6
Lhs = Lhe + 3.34e5

# compute some intermediate quantities for use below
Fql_turb = dharma_snds['Fqc_turb'].data+dharma_snds['Fqr_turb'].data
Fqi_turb = dharma_snds['Fqic_turb'].data+dharma_snds['Fqif_turb'].data+dharma_snds['Fqid_turb'].data
wql_tot = 0.5*(Fql_turb[:,0:nz]+Fql_turb[:,1:nz+1])/dharma_snds['rhobar'].data+dharma_snds['WQL'].data
wqi_tot = 0.5*(Fqi_turb[:,0:nz]+Fqi_turb[:,1:nz+1])/dharma_snds['rhobar'].data+dharma_snds['WQI'].data
PFql = dharma_snds['PFqc'].data+dharma_snds['PFqr'].data
PFqi = dharma_snds['PFqic'].data+dharma_snds['PFqif'].data+dharma_snds['PFqid'].data
wpfl = 0.5*(PFql[:,0:nz]+PFql[:,1:nz+1])*-1./3600./dharma_snds['rhobar'].data
wpfi = 0.5*(PFqi[:,0:nz]+PFqi[:,1:nz+1])*-1./3600./dharma_snds['rhobar'].data
Flwd = dharma_snds['Flw_dn'].data
Flwu = dharma_snds['Flw_up'].data
Fnlw = Flwu - Flwd
PFqc = dharma_snds['PFqc'].data 
PFqr = dharma_snds['PFqr'].data 
if do_ice:
    PFqic = dharma_snds['PFqic'].data 
    PFqif = dharma_snds['PFqif'].data 
    PFqid = dharma_snds['PFqid'].data 
if do_progNa:
    ssa_sfc = (dharma_snds['Sna_1_sfc'].data[:,0]+dharma_snds['Sna_2_sfc'].data[:,0]+dharma_snds['Sna_3_sfc'].data[:,0])*dharma_snds['zw'].data[1]

# append new variables to the data structure
dharma_snds = dharma_snds.assign(theta = dummy_snd + (dharma_snds['th'].data+1)*dharma_snds.theta_00)
dharma_snds = dharma_snds.assign(pi = dummy_snd + dharma_snds['T'].data/dharma_snds['theta'].data)
dharma_snds = dharma_snds.assign(pressure = dummy_snd + np.power(dharma_snds['pi'].data,7./2)*np.power(10.,5))
dharma_snds = dharma_snds.assign(PF = dummy_snd + 0.5*(PFqc[:,0:nz]+PFqc[:,1:nz+1]) + 0.5*(PFqr[:,0:nz]+PFqr[:,1:nz+1]))
dharma_snds = dharma_snds.assign(nlcic = dummy_snd + dharma_snds['nc_cld'].data*1.e6/dharma_snds['rhobar'].data)
if do_ice:
    dharma_snds = dharma_snds.assign(PFi = dummy_snd + 0.5*(PFqic[:,0:nz]+PFqic[:,1:nz+1]) + 0.5*(PFqif[:,0:nz]+PFqif[:,1:nz+1]) 
                                     + 0.5*(PFqid[:,0:nz]+PFqid[:,1:nz+1]) )
    dharma_snds['PF'] += dharma_snds['PFi']
#    if 'NEW_FROM_ANDY' in dharma_snds:
#         dharma_snds = dharma_snds.assign(niic = dummy_snd + dharma_snds['NEW_FROM_ANDY'].data*1.e6/dharma_snds['rhobar'].data)
dharma_snds = dharma_snds.assign(uw_zt = dummy_snd + 0.5*(dharma_snds['txz_tot'].data[:,0:nz]+dharma_snds['txz_tot'].data[:,1:nz+1]))
dharma_snds = dharma_snds.assign(vw_zt = dummy_snd + 0.5*(dharma_snds['tyz_tot'].data[:,0:nz]+dharma_snds['tyz_tot'].data[:,1:nz+1]))
dharma_snds = dharma_snds.assign(w2_zt = dummy_snd + 0.5*(dharma_snds['w2'].data[:,0:nz]+dharma_snds['w2'].data[:,1:nz+1]))
dharma_snds = dharma_snds.assign(wth_zt = dummy_snd + 0.5*(dharma_snds['qhz_tot'].data[:,0:nz] + 
                                        dharma_snds['qhz_tot'].data[:,1:nz+1])) #*dharma_snds.theta_00)
dharma_snds = dharma_snds.assign(wthli_zt = dummy_snd + dharma_snds['wth_zt'].data
                    - wql_tot*Lhe/(dharma_snds['pi'].data*cp) - wqi_tot*Lhs/(dharma_snds['pi'].data*cp)
                    - wpfl*Lhe/(dharma_snds['pi'].data*cp) - wpfi*Lhs/(dharma_snds['pi'].data*cp))                                 
dharma_snds = dharma_snds.assign(wqv_zt = dummy_snd + 0.5*(dharma_snds['qqz_tot'].data[:,0:nz]+dharma_snds['qqz_tot'].data[:,1:nz+1]))
dharma_snds = dharma_snds.assign(wqt_zt = dummy_snd + dharma_snds['wqv_zt'].data + wql_tot + wqi_tot + wpfl + wpfi)
dharma_snds = dharma_snds.assign(LWdn = dummy_snd + 0.5*(Flwd[:,0:nz]+Flwd[:,1:nz+1]))
dharma_snds = dharma_snds.assign(LWup = dummy_snd + 0.5*(Flwu[:,0:nz]+Flwu[:,1:nz+1]))
dharma_snds = dharma_snds.assign(HRlw = dummy_snd + 0.5*(Fnlw[:,0:nz]+Fnlw[:,1:nz+1])/dz/dharma_snds['rhobar'].data)
dharma_snds = dharma_snds.assign(dth_micro = dummy_snd + dharma_snds['Sth_micro'].data + dharma_snds['Sth_cond'].data)
dharma_snds = dharma_snds.assign(dq_micro = dummy_snd + dharma_snds['Sqv_micro'].data + dharma_snds['Sqv_cond'].data)

dharma_snds

<xarray.Dataset>
Dimensions:         (zt: 159, zw: 160, time: 121)
Coordinates:
  * zt              (zt) float32 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
  * zw              (zw) float32 0.0 20.0 45.0 75.0 ... 6.89e+03 6.95e+03 7e+03
  * time            (time) datetime64[ns] 2020-03-12T22:00:00 ... 2020-03-13T...
Data variables: (12/407)
    jact            (time, zt) float32 0.4543 0.5679 0.6814 ... 1.363 1.136
    jacw            (time, zw) float32 0.3975 0.5111 0.6246 ... 1.59 1.249 1.704
    rhobar          (time, zt) float32 1.399 1.396 1.391 ... 0.5989 0.594 0.5901
    u               (time, zt) float32 3.225 3.454 3.719 ... 4.824 4.85 4.877
    u2              (time, zt) float32 5.199 5.965 6.917 ... 11.64 11.77 11.9
    Su_rk           (time, zt) float32 0.0 0.0 0.0 ... 0.0003689 0.0003783
    ...              ...
    wqt_zt          (time, zt) float32 2.243e-06 1.433e-06 ... 1.155e-11
    LWdn            (time, zt) float32 122.5 122.4 122.2 ... 42.67 42.22 41.91
    LWup            (time, zt) float32 211.1 211.2 211.2 ... 205.0 204.7 204.5
    HRlw            (time, zt) float32 3.166 2.544 2.132 ... 3.389 4.559 5.51
    dth_micro       (time, zt) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    dq_micro        (time, zt) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    nx:        256
    ny:        256
    nz:        159
    L_x:       25600.0
    L_y:       25600.0
    H:         7000.0
    theta_00:  247.95700073242188

### Read domain-mean scalars

In [6]:
for index, elem in enumerate(my_outdirs):
    input_filename = my_rundir + elem + '/dharma.scalars.cdf'
    print(input_filename)
    if index==0:
        dharma_scas = xr.open_dataset(input_filename)
        dharma_scas['time'] = pd.to_datetime(dharma_scas['time'].values, unit='s', origin=pd.Timestamp(start_dtime))
        dharma_scas = dharma_scas.resample(time="600S",closed="right",label="right").mean()
    else:
        dharma_scas2 = xr.open_dataset(input_filename)
        dharma_scas2['time'] = pd.to_datetime(dharma_scas2['time'].values, unit='s', origin=pd.Timestamp(start_dtime))
        dharma_scas2 = dharma_scas2.resample(time="600S",closed="right",label="right").mean()
        dharma_scas = xr.concat([dharma_scas,dharma_scas2],dim='time')
dharma_scas = dharma_scas.drop_duplicates('time',keep='first')
dharma_scas

/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/0-2h/dharma.scalars.cdf
/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/02h-16h20m/dharma.scalars.cdf
/data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/16h20m-20h/dharma.scalars.cdf


<xarray.Dataset>
Dimensions:                     (time: 121)
Coordinates:
  * time                        (time) datetime64[ns] 2020-03-12T22:00:00 ......
Data variables: (12/111)
    iwp                         (time) float32 0.0 0.0 0.0 ... 328.5 391.6 468.9
    rms_iwp                     (time) float32 0.0 0.0 0.0 ... 417.3 598.7 828.4
    iwpf                        (time) float32 0.0 0.0 0.0 ... 243.8 273.9 302.2
    iwpd                        (time) float32 0.0 0.0 0.0 ... 53.08 87.69 136.0
    iwpc                        (time) float32 0.0 0.0 0.0 ... 31.62 30.03 30.67
    max_qi                      (time) float32 0.0 0.0 0.0 ... 0.001326 0.001406
    ...                          ...
    predictorstep_call          (time) float32 0.25 97.62 ... 4.425e+03
    setfirstphi_call            (time) float32 0.25 0.25 0.25 ... 0.0 0.0 0.0
    forwardeuler_call           (time) float32 0.0 0.75 1.625 ... 15.25 15.75
    compsadvect_call            (time) float32 0.0 48.88 ... 1.558e+03 1.633e+03
    processsources_call         (time) float32 8.75 127.5 ... 6.157e+03
    computepressureupdate_call  (time) float32 0.0 5.875 10.12 ... 154.2 160.3

In [7]:
# calculate some additional variables requested
dummy_sca = dharma_scas['lwp']*0.
dharma_scas = dharma_scas.assign(Psurf = dummy_sca + dharma_params['sounding'].Psurf*100.)
dharma_scas = dharma_scas.assign(opd_tot = dummy_sca + dharma_scas['opd_drops'].data + dharma_scas['opd_ice'].data)
dharma_scas = dharma_scas.assign(LWdnSFC = dummy_sca + dharma_snds['Flw_dn'].data[:,0])
dharma_scas = dharma_scas.assign(LWupSFC = dummy_sca + dharma_snds['Flw_up'].data[:,0])
dharma_scas = dharma_scas.assign(avg_precip_ice = dummy_sca + dharma_scas['avg_precip'].data 
                                 - dharma_snds['PFqc'].data[:,0] - dharma_snds['PFqr'].data[:,0])
dharma_scas = dharma_scas.assign(ssaf = dummy_sca + ssa_sfc)
dharma_scas

<xarray.Dataset>
Dimensions:                     (time: 121)
Coordinates:
  * time                        (time) datetime64[ns] 2020-03-12T22:00:00 ......
Data variables: (12/117)
    iwp                         (time) float32 0.0 0.0 0.0 ... 328.5 391.6 468.9
    rms_iwp                     (time) float32 0.0 0.0 0.0 ... 417.3 598.7 828.4
    iwpf                        (time) float32 0.0 0.0 0.0 ... 243.8 273.9 302.2
    iwpd                        (time) float32 0.0 0.0 0.0 ... 53.08 87.69 136.0
    iwpc                        (time) float32 0.0 0.0 0.0 ... 31.62 30.03 30.67
    max_qi                      (time) float32 0.0 0.0 0.0 ... 0.001326 0.001406
    ...                          ...
    Psurf                       (time) float32 9.954e+04 9.954e+04 ... 9.954e+04
    opd_tot                     (time) float32 0.0 0.0 0.0 ... 11.66 11.19 10.52
    LWdnSFC                     (time) float32 122.5 122.5 122.5 ... 258.1 253.3
    LWupSFC                     (time) float32 211.0 211.0 211.0 ... 339.6 338.4
    avg_precip_ice              (time) float32 0.0 0.0 0.0 ... 0.162 0.2128
    ssaf                        (time) float32 1.565e+06 1.026e+06 ... 9.034e+05

## Prepare output file in DEPHY format

### Read requested variables list

Variable description, naming, units, and dimensions.

In [8]:
# read list of requested variables
vars_mean_list = pd.read_excel('https://docs.google.com/spreadsheets/d/1Vl8jYGviet7EtXZuQiitrx4NSkV1x27aJAhxxjBb9zI/export?gid=0&format=xlsx',
                              sheet_name='Mean')

pd.set_option('display.max_rows', None)
vars_mean_list

,standard_name,variable_id,units,dimensions,"comment (10-min average reported at endpoints, green=minimum)"
0,time,time,s,–,"dimension, seconds since 2020-03-12 18:00:00"
1,height,zf,m,–,"dimension, altitude of mid-level points above ..."
2,layer_top_height,ze,m,–,"dimension, altitude of layer top points above ..."
3,surface_pressure,ps,Pa,time,–
4,surface_temperature,ts,K,time,–
5,surface_friction_velocity,ustar,m s-1,time,–
6,surface_roughness_length_for_momentum_in_air,z0,m,time,–
7,surface_roughness_length_for_heat_in_air,z0h,m,time,–
8,surface_roughness_length_for_humidity_in_air,z0q,m,time,–
9,surface_upward_sensible_heat_flux,hfss,W m-2,time,–


### Match DHARMA variables to requested outputs

Expand the table to include columns that indicate DHARMA model variable names and any conversion factor.

In [9]:
# drop comments
vars_mean_list = vars_mean_list.drop(columns='comment (10-min average reported at endpoints, green=minimum)')

# add columns to contain model output name and units conversion factors
vars_mean_list = vars_mean_list.assign(model_name='missing data',conv_factor=1.0)

In [10]:
# identify requested variables with only time dimension
vars_mean_scas = vars_mean_list[vars_mean_list['dimensions']=='time']

# match to DHARMA variable names and specify conversion factors
for index in vars_mean_scas.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='surface_pressure': 
        vars_mean_list.model_name.iat[index] = 'Psurf'
    if standard_name=='surface_temperature': 
        vars_mean_list.model_name.iat[index] = 'avg_T_sfc'
    if standard_name=='surface_friction_velocity': 
        vars_mean_list.model_name.iat[index] = 'avg_ustar'
    if standard_name=='surface_roughness_length_for_momentum_in_air':
        vars_mean_list.model_name.iat[index] = 'avg_z0'
    if standard_name=='surface_roughness_length_for_heat_in_air':
        vars_mean_list.model_name.iat[index] = 'avg_z0h'
    if standard_name=='surface_roughness_length_for_humidity_in_air':
        # same as roughness length for heat in DHARMA
        vars_mean_list.model_name.iat[index] = 'avg_z0h'
    if standard_name=='surface_upward_sensible_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'avg_T_flx'
    if standard_name=='surface_upward_latent_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'avg_qv_flx'
    if standard_name=='obukhov_length': 
        vars_mean_list.model_name.iat[index] = 'avg_obk'
    if standard_name=='atmosphere_mass_content_of_liquid_cloud_water': 
        vars_mean_list.model_name.iat[index] = 'cwp'
        vars_mean_list.conv_factor.iat[index] = 1/1000.
    if standard_name=='atmosphere_mass_content_of_rain_water': 
        vars_mean_list.model_name.iat[index] = 'rwp'
        vars_mean_list.conv_factor.iat[index] = 1/1000.
    if do_ice:
        if standard_name=='atmosphere_mass_content_of_ice_water': 
            vars_mean_list.model_name.iat[index] = 'iwp'
            vars_mean_list.conv_factor.iat[index] = 1/1000.
    if standard_name=='cloud_area_fraction': 
        vars_mean_list.model_name.iat[index] = 'colf_opd'
    if standard_name=='optical_depth': 
        vars_mean_list.model_name.iat[index] = 'opd_tot'
    if standard_name=='optical_depth_of_liquid_cloud': 
        vars_mean_list.model_name.iat[index] = 'opd_cloud'
    if standard_name=='precipitation_flux_at_surface': 
        vars_mean_list.model_name.iat[index] = 'avg_precip'
        vars_mean_list.conv_factor.iat[index] = 1/3600.
    if do_ice:
        if standard_name=='precipitation_flux_at_surface_in_ice_phase': 
            vars_mean_list.model_name.iat[index] = 'avg_precip_ice'
            vars_mean_list.conv_factor.iat[index] = 1/3600.
    if standard_name=='optical_depth_of_cloud_droplets': 
        vars_mean_list.model_name.iat[index] = 'opd_cloud'
    if standard_name=='toa_outgoing_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'LWupTOA'
    if standard_name=='surface_downwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'LWdnSFC'  
    if standard_name=='surface_upwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'LWupSFC'
    if do_progNa:
        if standard_name=='surface_sea_spray_number_flux': 
            vars_mean_list.model_name.iat[index] = 'ssaf'

In [11]:
# identify requested variables with time and vertical dimensions
vars_mean_snds = vars_mean_list[vars_mean_list['dimensions']=='time, height']

# match to DHARMA variable names and specify conversion factors
for index in vars_mean_snds.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='air_pressure': 
        vars_mean_list.model_name.iat[index] = 'pressure'
    if standard_name=='eastward_wind': 
        vars_mean_list.model_name.iat[index] = 'u'
    if standard_name=='northward_wind': 
        vars_mean_list.model_name.iat[index] = 'v'
    if standard_name=='air_dry_density': 
        vars_mean_list.model_name.iat[index] = 'rhobar'
    if standard_name=='air_temperature': 
        vars_mean_list.model_name.iat[index] = 'T'
    if standard_name=='water_vapor_mixing_ratio': 
        vars_mean_list.model_name.iat[index] = 'qv'
    if standard_name=='relative_humidity': 
        vars_mean_list.model_name.iat[index] = 'RH'
        vars_mean_list.conv_factor.iat[index] = 1/100.
    if standard_name=='relative_humidity_over_ice': 
        vars_mean_list.model_name.iat[index] = 'RHI'
        vars_mean_list.conv_factor.iat[index] = 1/100.
    if standard_name=='air_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'theta'
    if standard_name=='specific_turbulent_kinetic_energy_resolved': 
        vars_mean_list.model_name.iat[index] = 'tkeavg'
    if standard_name=='specific_turbulent_kinetic_energy_sgs': 
        vars_mean_list.model_name.iat[index] = 'tke_smag'
    if standard_name=='mass_mixing_ratio_of_cloud_liquid_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'qc'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'qr'
    if do_ice:
        if standard_name=='mass_mixing_ratio_of_cloud_ice_in_air': 
            vars_mean_list.model_name.iat[index] = 'qic'
        if standard_name=='mass_mixing_ratio_of_snow_in_air': 
            vars_mean_list.model_name.iat[index] = 'qif'
        if standard_name=='mass_mixing_ratio_of_graupel_in_air': 
            vars_mean_list.model_name.iat[index] = 'qid'
    if standard_name=='number_of_liquid_cloud_droplets_in_air': 
        vars_mean_list.model_name.iat[index] = 'nc'
    if standard_name=='number_of_rain_drops_in_air': 
        vars_mean_list.model_name.iat[index] = 'nr'
    if do_ice:
        if standard_name=='number_of_cloud_ice_crystals_in_air': 
            vars_mean_list.model_name.iat[index] = 'nic'
        if standard_name=='number_of_snow_crystals_in_air': 
            vars_mean_list.model_name.iat[index] = 'nif'
        if standard_name=='number_of_graupel_crystals_in_air': 
            vars_mean_list.model_name.iat[index] = 'nid'    
    if do_progNa:
        if standard_name=='number_of_total_aerosol_mode1': 
            vars_mean_list.model_name.iat[index] = 'na_1'
        if standard_name=='number_of_total_aerosol_mode2': 
            vars_mean_list.model_name.iat[index] = 'na_2'
        if standard_name=='number_of_total_aerosol_mode3': 
            vars_mean_list.model_name.iat[index] = 'na_3'
    if standard_name=='number_of_liquid_cloud_droplets_in_cloud': 
        vars_mean_list.model_name.iat[index] = 'nlcic'
#    if standard_name=='disspation_rate_of_turbulent_kinetic_energy': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='zonal_momentum_flux': 
        vars_mean_list.model_name.iat[index] = 'uw_zt'
    if standard_name=='meridional_momentum_flux': 
        vars_mean_list.model_name.iat[index] = 'vw_zt'
    if standard_name=='variance_of_upward_air_velocity': 
        vars_mean_list.model_name.iat[index] = 'w2_zt'
    if standard_name=='vertical_flux_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'wth_zt'
    if standard_name=='vertical_flux_liquid_ice_water_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'wthli_zt'
    if standard_name=='vertical_flux_water_vapor': 
        vars_mean_list.model_name.iat[index] = 'wqv_zt'
    if standard_name=='vertical_flux_total_water': 
        vars_mean_list.model_name.iat[index] = 'wqt_zt'
    if standard_name=='area_fraction_of_liquid_cloud': 
        vars_mean_list.model_name.iat[index] = 'cloud_f'
    if standard_name=='precipitation_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'PF'
        vars_mean_list.conv_factor.iat[index] = 1/(3600.)
    if standard_name=='precipitation_flux_in_air_in_ice_phase': 
        vars_mean_list.model_name.iat[index] = 'PFi'
        vars_mean_list.conv_factor.iat[index] = 1/(3600.)
    if standard_name=='downwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'LWdn'
    if standard_name=='upwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'LWup'
    if standard_name=='tendency_of_air_potential_temperature_due_to_radiative_heating': 
        vars_mean_list.model_name.iat[index] = 'Srad'
        vars_mean_list.conv_factor.iat[index] = 1/3600.
    if standard_name=='tendency_of_air_potential_temperature_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'dth_micro'
        vars_mean_list.conv_factor.iat[index] = 1/3600.
    if standard_name=='tendency_of_air_potential_temperature_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'Sth_adv'
        vars_mean_list.conv_factor.iat[index] = 1/3600.
    if standard_name=='tendency_of_water_vapor_mixing_ratio_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'dq_micro'
        vars_mean_list.conv_factor.iat[index] = 1/3600.*1/1000.
    if standard_name=='tendency_of_water_vapor_mixing_ratio_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'Sqv_adv'
    if do_ice:
        if standard_name=='tendency_of_ice_number_due_to_heterogeneous_freezing': 
            vars_mean_list.model_name.iat[index] = 'Sice_het'
        if standard_name=='tendency_of_ice_number_due_to_secondary_ice_production': 
            vars_mean_list.model_name.iat[index] = 'Sice_sec'
        if standard_name=='tendency_of_ice_number_due_to_homogeneous_freezing': 
            vars_mean_list.model_name.iat[index] = 'Sice_hom'

vars_mean_list[3:] # echo variables (first rows are dimensions)

,standard_name,variable_id,units,dimensions,model_name,conv_factor
3,surface_pressure,ps,Pa,time,Psurf,1.000000e+00
4,surface_temperature,ts,K,time,avg_T_sfc,1.000000e+00
5,surface_friction_velocity,ustar,m s-1,time,avg_ustar,1.000000e+00
6,surface_roughness_length_for_momentum_in_air,z0,m,time,avg_z0,1.000000e+00
7,surface_roughness_length_for_heat_in_air,z0h,m,time,avg_z0h,1.000000e+00
8,surface_roughness_length_for_humidity_in_air,z0q,m,time,avg_z0h,1.000000e+00
9,surface_upward_sensible_heat_flux,hfss,W m-2,time,avg_T_flx,1.000000e+00
10,surface_upward_latent_heat_flux,hfls,W m-2,time,avg_qv_flx,1.000000e+00
11,obukhov_length,ol,m,time,avg_obk,1.000000e+00
12,atmosphere_mass_content_of_liquid_cloud_water,lwpc,kg m-2,time,cwp,1.000000e-03


## Create DEPHY output file

Write a single file to contain all domain-mean scalar and profile outputs. This code expects the write directory to be pre-existing (already created by the user). In the case that this output will be committed to the comble-mip GitHub repository, see above "Specify directory locations".

In [12]:
# create DEPHY output file
dephy_filename = './' + my_gitdir + my_outfile
if os.path.exists(dephy_filename):
    os.remove(dephy_filename)
    print('The file ' + dephy_filename + ' has been deleted successfully')    
dephy_file = Dataset(dephy_filename,mode='w',format='NETCDF3_CLASSIC')

# create global attributes
dephy_file.title='DHARMA LES results for COMBLE-MIP case: fixed Nd and Ni'
dephy_file.reference='https://github.com/ARM-Development/comble-mip'
dephy_file.authors='Ann Fridlind (ann.fridlind@nasa.gov) and Florian Tornow (florian.tornow@nasa.gov)'
dephy_file.source=input_filename
dephy_file.version=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dephy_file.format_version='DEPHY SCM format version 1.6'
dephy_file.script='convert_DHARMA_LES_output_to_dephy_format.ipynb'
dephy_file.startDate=start_dtime
dephy_file.force_geo=1
dephy_file.surfaceType='ocean (after spin-up)'
dephy_file.surfaceForcing='ts (after spin-up)'
dephy_file.lat=str(dharma_params['Coriolis'].lat) + ' deg N'
dephy_file.dx=str(dharma_params['geometry'].L_x/dharma_params['geometry'].nx) + ' m'
dephy_file.dy=str(dharma_params['geometry'].L_y/dharma_params['geometry'].ny) + ' m'
dephy_file.dz='see zf variable'
dephy_file.nx=dharma_params['geometry'].nx
dephy_file.ny=dharma_params['geometry'].ny
dephy_file.nz=dharma_params['geometry'].nz

# create dimensions
nz = dharma_snds.dims['zt']
zf = dephy_file.createDimension('zf', nz)
zf = dephy_file.createVariable('zf', np.float64, ('zf',))
zf.units = 'm'
zf.long_name = 'height'
zf[:] = dharma_snds['zt'].data

ze = dephy_file.createDimension('ze', nz)
ze = dephy_file.createVariable('ze', np.float64, ('ze',))
ze.units = 'm'
ze.long_name = 'layer_top_height'
ze[:] = dharma_snds['zw'].data[1:]

nt = dharma_snds.dims['time']
time = dephy_file.createDimension('time', nt)
time = dephy_file.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + dephy_file.startDate
time.long_name = 'time'
# find time step and build time in seconds
delta_t = (dharma_snds['time'].data[1]-dharma_snds['time'].data[0])/np.timedelta64(1, "s")
time[:] = np.arange(nt)*delta_t

# create and fill variables
for index in vars_mean_list.index[2:]:
    std_name = vars_mean_list.standard_name.iat[index]
#   print(std_name) # debug
    var_name = vars_mean_list.variable_id.iat[index]
    mod_name = vars_mean_list.model_name.iat[index]
    c_factor = vars_mean_list.conv_factor.iat[index]
    if vars_mean_list.dimensions.iat[index]=='time':
        new_sca = dephy_file.createVariable(var_name, np.float64, ('time'))
        new_sca.units = vars_mean_list.units.iat[index]
        new_sca.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data' and mod_name in dharma_scas:
            new_sca[:] = dharma_scas[mod_name].data*c_factor
    if vars_mean_list.dimensions.iat[index]=='time, height':
        new_snd = dephy_file.createVariable(var_name, np.float64, ('time','zf'))
        new_snd.units = vars_mean_list.units.iat[index]
        new_snd.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data' and mod_name in dharma_snds: 
            new_snd[:] = dharma_snds[mod_name].data*c_factor

print(dephy_file)
dephy_file.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: DHARMA LES results for COMBLE-MIP case: fixed Nd and Ni
    reference: https://github.com/ARM-Development/comble-mip
    authors: Ann Fridlind (ann.fridlind@nasa.gov) and Florian Tornow (florian.tornow@nasa.gov)
    source: /data/home/fridlind/dharma/devel/case0313_progn_ice25_miz_cfmip_dx100/16h20m-20h/dharma.scalars.cdf
    version: 2024-05-10 20:52:20
    format_version: DEPHY SCM format version 1.6
    script: convert_DHARMA_LES_output_to_dephy_format.ipynb
    startDate: 2020-03-12 22:00:00.0
    force_geo: 1
    surfaceType: ocean (after spin-up)
    surfaceForcing: ts (after spin-up)
    lat: 73.0 deg N
    dx: 100.0 m
    dy: 100.0 m
    dz: see zf variable
    nx: 256
    ny: 256
    nz: 159
    dimensions(sizes): zf(159), ze(159), time(121)
    variables(dimensions): float64 zf(zf), float64 ze(ze), float64 time(time), float64 ps(time), float64 ts(time), float64 ustar(ti

### Check output file

In [13]:
dephy_filename

'./../../output_les/dharma/devel/DHARMA_Lx25km_dx100_ProgNa.nc'

In [14]:
dephy_check = xr.open_dataset(dephy_filename)
dephy_check

<xarray.Dataset>
Dimensions:         (zf: 159, ze: 159, time: 121)
Coordinates:
  * zf              (zf) float64 10.0 32.5 60.0 ... 6.85e+03 6.92e+03 6.975e+03
  * ze              (ze) float64 20.0 45.0 75.0 ... 6.89e+03 6.95e+03 7e+03
  * time            (time) datetime64[ns] 2020-03-12T22:00:00 ... 2020-03-13T...
Data variables: (12/71)
    ps              (time) float64 ...
    ts              (time) float64 ...
    ustar           (time) float64 ...
    z0              (time) float64 ...
    z0h             (time) float64 ...
    z0q             (time) float64 ...
    ...              ...
    dna_micro_warm  (time, zf) float64 ...
    dna_micro_cold  (time, zf) float64 ...
    dna_turb        (time, zf) float64 ...
    dni_het         (time, zf) float64 ...
    dni_sip         (time, zf) float64 ...
    dni_hom         (time, zf) float64 ...
Attributes: (12/18)
    title:           DHARMA LES results for COMBLE-MIP case: fixed Nd and Ni
    reference:       https://github.com/ARM-Development/comble-mip
    authors:         Ann Fridlind (ann.fridlind@nasa.gov) and Florian Tornow ...
    source:          /data/home/fridlind/dharma/devel/case0313_progn_ice25_mi...
    version:         2024-05-10 20:52:20
    format_version:  DEPHY SCM format version 1.6
    ...              ...
    dx:              100.0 m
    dy:              100.0 m
    dz:              see zf variable
    nx:              256
    ny:              256
    nz:              159